In [2]:
#  시스템콜 이름 -> 시스템콜 번호 딕셔너리 만들기
import subprocess
sysname_convert = dict()
sysnum_convert = dict()
with open("/tmp/syscall_64.txt","r") as f:
    for line in f.readlines():
        sysnum = line.strip().split()[0] # sysnum
        if len(line.strip().split()) == 4:
            sysname = line.strip().split()[3] # sysname
            if "compat_sys_" in sysname:
                sysname = sysname.replace("compat_sys_","")
            elif "sys_" in sysname:
                sysname = sysname.replace("sys_","")
            if "stub_x32_" in sysname:
                sysname = sysname.replace("stub_x32_","")
            if "stub_" in sysname:
                sysname = sysname.replace("stub_","")
        else:
            sysname = line.strip().split()[2] # sysname
        # x86 기준, x86 시스템콜에 없는 번호는 x32 기준
        if not sysname in sysname_convert.keys():
            sysname_convert[sysname] = sysnum
            sysnum_convert[sysnum] = sysname

In [3]:
# 기존에 없는 시스템콜 처리
sysname_convert["socketcall"] = "437"
sysnum_convert["437"] = "socketcall"
sysname_convert["vm86old"] = "436"
sysnum_convert["436"] = "vm86old"

# Benign Image

In [4]:
import subprocess

# 모든 benign syscall sequence load
seqList = dict()
imgList = {"httpd", "mongodb", "mysql", "nginx", "redis", "mariadb", "node", "tomcat", "gcc", "openjdk", "gzip", "bzip2", "ghostscript", "lowriter", "qalc"}

for imgName in imgList:
    seqs = subprocess.check_output(f"find ../data/parsing/{imgName}/*.txt", shell=True).decode().strip().split()
    for seq in seqs:
        case = seq.replace(f"../data/parsing/{imgName}/","").replace(".txt","")
        with open(seq, "r") as f:
            syslist = f.readlines()
            syslist = list(map(lambda x: x.strip(), syslist))
            seqList[f"{imgName}_{case}"]=syslist
        
print(len(seqList))

46


In [5]:
# 시스템콜 번호으로 바꾸기
seqNumList = dict()

for case, seq in seqList.items():
    # 변환
    num_seq = list()
    for sys_name in seq:
        num_seq.append(sysname_convert[sys_name])
    seqNumList[case]=num_seq

## System Call Set 분석

In [6]:
# System Call Set 분석
b_sysSet = dict()
for imgName in imgList:
    b_sysSet[imgName] = set()

for case, seq in seqNumList.items():
    imgName = case.split("_")[0]
    for syscall in seq:
        b_sysSet[imgName].add(syscall)
        
b_sysSet_total = set()
for imgName in imgList:
    print(f"image {imgName} syscall set #: {len(b_sysSet[imgName])}")
    b_sysSet_total = b_sysSet_total | b_sysSet[imgName]
print(f"total syscall set #: {len(b_sysSet_total)}")

image gcc syscall set #: 38
image httpd syscall set #: 21
image qalc syscall set #: 42
image gzip syscall set #: 20
image mysql syscall set #: 39
image mongodb syscall set #: 45
image mariadb syscall set #: 22
image node syscall set #: 13
image redis syscall set #: 14
image bzip2 syscall set #: 17
image ghostscript syscall set #: 38
image tomcat syscall set #: 25
image lowriter syscall set #: 76
image nginx syscall set #: 14
image openjdk syscall set #: 57
total syscall set #: 107


# Exploit Code N-gram

In [7]:
import pickle

## Load pickle
# with open("/opt/output/perm/analysis/soft_dedup_ngramdf.pkl","rb") as f:
with open("soft_dedup_ngramdf.pkl","rb") as f:
    m_ngram = pickle.load(f)
m_ngram

,N,Ngram_syscall,num_of_exploit,EIDs
0,61,"[41, 248, 248, 248, 54, 0, 3, 5, 9, 10, 12, 13...",2,"[40871, 47170]"
1,49,"[230, 0, 3, 5, 9, 10, 12, 13, 14, 17, 56, 218,...",2,"[40871, 47170]"
2,45,"[12, 12, 5, 0, 257, 4, 12, 12, 5, 0, 257, 4, 1...",2,"[47163, 50541]"
3,43,"[9, 0, 3, 5, 9, 10, 12, 13, 14, 17, 56, 218, 2...",2,"[40616, 43127]"
4,43,"[12, 12, 5, 0, 257, 4, 12, 12, 5, 0, 257, 4, 1...",2,"[47163, 50541]"
...,...,...,...,...
473,2,"[257, 0]",16,"[15024, 15774, 3, 35370, 40616, 40871, 42275, ..."
474,2,"[8, 0]",8,"[12, 20979, 27297, 3, 43127, 45553, 50808, 9083]"
475,2,"[3, 59]",9,"[39230, 40871, 43418, 47170, 6851, 9479, 9542]"
476,2,"[8, 5]",4,"[12, 3, 43127, 50541]"


# Exploit(106)과 Image Set 분석

## Exploit(106) Syscall Set

In [8]:
# 모든 익스플로잇 path laod
import subprocess
import json

pathFile = subprocess.check_output(f"find ../data/path/*.json", shell=True).decode().strip().split()
EIDList = list(map(lambda x: x.replace("../data/path/","").replace(".json",""), pathFile))
pathList = dict() # 익스플로잇 코드 path dictionary의 dictionary - pathList[EID][path] = path 파일 정보
for EID in EIDList:
    with open(f"../data/path/{EID}.json", "r") as f:
        pathList[EID] = dict()
        pathList[EID]["path"] = json.load(f)

In [9]:
# dict {EID: all path list} 만들어 주기
eid_all_path_dict = dict()
for EID, pathset in pathList.items():
    eid_all_path_dict[EID] = list()
    for userfunc, pathset in pathset['path'].items():
        if not len(pathset) == 0:
            eid_all_path_dict[EID].extend(pathset)

In [10]:
# 중복 제거
eid_all_path_set = dict()
for EID, pathset in eid_all_path_dict.items():
    eid_all_path_set[EID] = set()
    for path in pathset:
        eid_all_path_set[EID].add(",".join(path))

In [11]:
m_sysSet = set()
for EID, pathset in eid_all_path_set.items():
    for path in pathset:
        if path:
            m_sysSet = m_sysSet | set(path.split(","))
len(m_sysSet)

140

In [12]:
def false_positive_set(m, b):
    falsePositive_sysSet = m & b
    print(f"false positive syscall #: {len(falsePositive_sysSet)}")
    tmp = set()
    for syscall in falsePositive_sysSet:
        tmp.add(sysnum_convert[syscall])
    print(tmp)

In [13]:
# all
print("="*10,"<all image>","="*10)
false_positive_set(m_sysSet, b_sysSet_total)
for imgName in imgList:
    print("="*10,f"<{imgName}>","="*10)
    false_positive_set(m_sysSet, b_sysSet[imgName])

========== <all image> ==========
false positive syscall #: 80
{'getpid', 'rt_sigreturn', 'getcwd', 'newfstatat', 'socket', 'clock_nanosleep', 'bind', 'munmap', 'recvfrom', 'setsockopt', 'set_tid_address', 'prctl', 'close', 'brk', 'listen', 'openat', 'epoll_wait', 'madvise', 'sched_yield', 'getuid', 'rmdir', 'sendmmsg', 'exit_group', 'shutdown', 'chown', 'read', 'clone', 'sigaltstack', 'epoll_ctl', 'sendfile64', 'execve', 'getegid', 'poll', 'geteuid', 'rename', 'link', 'rt_sigaction', 'futex', 'arch_prctl', 'setrlimit', 'nanosleep', 'mkdir', 'socketpair', 'rt_sigprocmask', 'newstat', 'ftruncate', 'getdents64', 'set_robust_list', 'getsockname', 'chmod', 'lseek', 'getsockopt', 'mprotect', 'unlink', 'prlimit64', 'ioctl', 'timerfd_settime', 'getppid', 'newfstat', 'sendto', 'statfs', 'write', 'exit', 'dup', 'newuname', 'umask', 'pread64', 'recvmsg', 'sendmsg', 'accept', 'wait4', 'open', 'access', 'mmap', 'pipe', 'gettid', 'readlink', 'connect', 'getgid', 'fcntl'}
========== <gcc> ==========

# Token 크기 별 Overlapped N-gram 개수

In [44]:
import pandas as pd

overlap_ngram = pd.DataFrame(columns=["N","nginx","httpd","tomcat","node","mongodb","mysql","mariadb","redis",
                                      "gcc","openjdk","gzip","bzip2","qalc","ghostscript","lowriter","total"])

for N in set(m_ngram["N"].to_list()):
    overlaptmp = {"N":N, "nginx":set(),"httpd":set(),"tomcat":set(),"node":set(),"mongodb":set(),"mysql":set(),
                  "mariadb":set(),"redis":set(),"gcc":set(),"openjdk":set(),"gzip":set(),"bzip2":set(),"qalc":set(),
                  "ghostscript":set(),"lowriter":set(),"total":set()}
    overlap_ngram = overlap_ngram.append(overlaptmp, ignore_index=True)
overlap_ngram = overlap_ngram.set_index("N")

for index, row in m_ngram.iterrows():
    for testName, seq in seqNumList.items():
        imgName = testName.split("_")[0]
        if ",".join(row.Ngram_syscall) in ",".join(seq):
            cur = overlap_ngram[imgName].loc[row.N]
            new = cur | {",".join(row.Ngram_syscall)}
            # new
            overlap_ngram.at[row.N,imgName] = new
            
overlap_ngram

,nginx,httpd,tomcat,node,mongodb,mysql,mariadb,redis,gcc,openjdk,gzip,bzip2,qalc,ghostscript,lowriter,total
N,,,,,,,,,,,,,,,,
2,"{3,3, 257,5, 1,3}","{0,8, 257,257, 257,9, 9,9, 9,257, 5,0, 3,0, 5,...","{3,3, 54,54, 1,0, 3,257, 8,1, 5,0, 3,0, 257,5,...","{3,3, 12,12, 8,1, 8,0}","{3,3, 3,257, 257,0, 5,9, 3,13, 1,3, 257,9, 4,0...","{5,3, 0,8, 9,11, 1,0, 3,257, 257,0, 5,9, 0,5, ...","{5,3, 3,3, 4,0, 257,1, 54,54, 257,0, 8,1, 0,5,...","{3,3, 54,54, 9,9, 257,0, 39,257, 9,257, 257,3,...","{5,3, 0,8, 3,3, 13,14, 5,9, 11,3, 3,13, 1,3, 1...","{5,3, 0,8, 3,3, 13,14, 3,257, 257,0, 5,9, 11,3...","{3,3, 9,9, 1,0, 12,12, 0,0, 8,1, 5,5, 1,3, 0,3}","{3,3, 9,9, 1,0, 12,12, 3,231, 0,0, 11,3, 8,1, ...","{0,8, 3,3, 13,14, 22,3, 3,257, 257,0, 5,9, 1,2...","{0,8, 3,3, 3,257, 257,0, 5,9, 11,3, 3,13, 1,3,...","{3,3, 79,21, 13,14, 5,9, 3,13, 13,13, 3,56, 10...",{}
3,"{5,1,3}","{9,9,9, 8,0,3}","{3,3,22, 257,5,0, 3,257,5, 5,0,3}","{3,3,22, 1,1,1}","{8,5,0, 257,257,0, 0,3,3, 0,0,0, 3,3,1, 257,5,...","{8,8,8, 8,5,0, 257,5,9, 0,0,0, 9,9,9, 257,5,0,...","{3,3,22, 257,0,3}","{3,3,22, 257,0,3, 54,54,54}","{8,8,8, 56,56,56, 0,8,8, 16,16,3, 1,1,1, 0,0,0...","{9,56,56, 8,8,8, 257,257,257, 257,0,3, 56,56,5...","{5,12,12, 12,12,1, 102,12,12, 1,1,1, 0,3,3, 0,...","{12,12,1, 1,1,1, 3,3,1, 0,0,0, 12,12,9, 9,9,9,...","{257,257,257, 56,56,56, 257,257,0, 0,8,8, 1,1,...","{8,8,8, 257,257,257, 0,8,8, 257,257,0, 257,12,...","{21,12,12, 56,56,56, 0,8,8, 1,1,1, 12,12,257, ...",{}
4,{},{},{},{},"{0,0,0,0, 0,0,0,3}","{9,9,9,9, 257,5,0,0, 0,0,0,3, 0,0,0,0, 9,3,257,0}",{},{},"{3,3,3,3, 12,12,12,12, 9,9,9,9, 0,0,0,3, 0,0,0,0}","{3,3,3,3, 12,12,12,12, 9,9,9,9, 0,0,0,3, 0,0,0,0}","{0,0,0,0, 12,12,12,12, 9,9,9,9}","{0,0,0,0, 12,12,12,12, 9,9,9,9}","{12,12,12,12, 9,9,9,9, 0,0,0,3, 0,0,0,0, 17,17...","{3,3,3,3, 9,9,9,9, 12,12,12,12, 3,12,12,257, 0...","{9,9,9,9, 3,12,12,257, 0,0,0,3, 0,0,0,0, 3,3,3...",{}
5,{},{},{},{},"{257,0,0,0,0, 0,0,0,0,3}","{5,9,9,9,9, 0,0,0,0,3}",{},{},"{9,9,9,9,9, 5,9,9,9,9, 0,0,0,0,3}","{9,9,9,9,9, 5,9,9,9,9, 0,0,0,0,3}",{},"{9,9,9,9,9}","{9,9,9,9,9, 5,9,9,9,9, 0,0,0,0,3}","{257,0,0,0,0, 0,0,0,0,3}","{9,9,9,9,9, 5,9,9,9,9, 257,0,0,0,0, 0,0,0,0,3}",{}
6,{},{},{},{},{},{},{},{},"{12,12,12,12,12,12}","{5,9,9,9,9,9}","{12,12,12,12,12,12}",{},"{5,9,9,9,9,9}","{12,12,12,12,12,12}","{12,12,12,12,12,12, 5,9,9,9,9,9}",{}
7,{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{}
8,{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{}
9,{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{}
10,{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{}


In [45]:
# total 구하기
for index, row in overlap_ngram.iterrows():
    tmpset = set()
    for imgName in imgList:
        tmpset = tmpset | overlap_ngram[imgName][index]
    overlap_ngram["total"][index] = tmpset
    
# pickle로 저장
overlap_ngram.to_pickle('overlap_ngram.pkl')
overlap_ngram

,nginx,httpd,tomcat,node,mongodb,mysql,mariadb,redis,gcc,openjdk,gzip,bzip2,qalc,ghostscript,lowriter,total
N,,,,,,,,,,,,,,,,
2,"{3,3, 257,5, 1,3}","{0,8, 257,257, 257,9, 9,9, 9,257, 5,0, 3,0, 5,...","{3,3, 54,54, 1,0, 3,257, 8,1, 5,0, 3,0, 257,5,...","{3,3, 12,12, 8,1, 8,0}","{3,3, 3,257, 257,0, 5,9, 3,13, 1,3, 257,9, 4,0...","{5,3, 0,8, 9,11, 1,0, 3,257, 257,0, 5,9, 0,5, ...","{5,3, 3,3, 4,0, 257,1, 54,54, 257,0, 8,1, 0,5,...","{3,3, 54,54, 9,9, 257,0, 39,257, 9,257, 257,3,...","{5,3, 0,8, 3,3, 13,14, 5,9, 11,3, 3,13, 1,3, 1...","{5,3, 0,8, 3,3, 13,14, 3,257, 257,0, 5,9, 11,3...","{3,3, 9,9, 1,0, 12,12, 0,0, 8,1, 5,5, 1,3, 0,3}","{3,3, 9,9, 1,0, 12,12, 3,231, 0,0, 11,3, 8,1, ...","{0,8, 3,3, 13,14, 22,3, 3,257, 257,0, 5,9, 1,2...","{0,8, 3,3, 3,257, 257,0, 5,9, 11,3, 3,13, 1,3,...","{3,3, 79,21, 13,14, 5,9, 3,13, 13,13, 3,56, 10...","{3,3, 79,21, 13,14, 5,9, 1,230, 3,13, 9,44, 13..."
3,"{5,1,3}","{9,9,9, 8,0,3}","{3,3,22, 257,5,0, 3,257,5, 5,0,3}","{3,3,22, 1,1,1}","{8,5,0, 257,257,0, 0,3,3, 0,0,0, 3,3,1, 257,5,...","{8,8,8, 8,5,0, 257,5,9, 0,0,0, 9,9,9, 257,5,0,...","{3,3,22, 257,0,3}","{3,3,22, 257,0,3, 54,54,54}","{8,8,8, 56,56,56, 0,8,8, 16,16,3, 1,1,1, 0,0,0...","{9,56,56, 8,8,8, 257,257,257, 257,0,3, 56,56,5...","{5,12,12, 12,12,1, 102,12,12, 1,1,1, 0,3,3, 0,...","{12,12,1, 1,1,1, 3,3,1, 0,0,0, 12,12,9, 9,9,9,...","{257,257,257, 56,56,56, 257,257,0, 0,8,8, 1,1,...","{8,8,8, 257,257,257, 0,8,8, 257,257,0, 257,12,...","{21,12,12, 56,56,56, 0,8,8, 1,1,1, 12,12,257, ...","{21,12,12, 56,56,56, 0,8,8, 1,1,1, 16,16,3, 12..."
4,{},{},{},{},"{0,0,0,0, 0,0,0,3}","{9,9,9,9, 257,5,0,0, 0,0,0,3, 0,0,0,0, 9,3,257,0}",{},{},"{3,3,3,3, 12,12,12,12, 9,9,9,9, 0,0,0,3, 0,0,0,0}","{3,3,3,3, 12,12,12,12, 9,9,9,9, 0,0,0,3, 0,0,0,0}","{0,0,0,0, 12,12,12,12, 9,9,9,9}","{0,0,0,0, 12,12,12,12, 9,9,9,9}","{12,12,12,12, 9,9,9,9, 0,0,0,3, 0,0,0,0, 17,17...","{3,3,3,3, 9,9,9,9, 12,12,12,12, 3,12,12,257, 0...","{9,9,9,9, 3,12,12,257, 0,0,0,3, 0,0,0,0, 3,3,3...","{9,9,9,9, 3,12,12,257, 257,5,0,0, 0,0,0,3, 0,0..."
5,{},{},{},{},"{257,0,0,0,0, 0,0,0,0,3}","{5,9,9,9,9, 0,0,0,0,3}",{},{},"{9,9,9,9,9, 5,9,9,9,9, 0,0,0,0,3}","{9,9,9,9,9, 5,9,9,9,9, 0,0,0,0,3}",{},"{9,9,9,9,9}","{9,9,9,9,9, 5,9,9,9,9, 0,0,0,0,3}","{257,0,0,0,0, 0,0,0,0,3}","{9,9,9,9,9, 5,9,9,9,9, 257,0,0,0,0, 0,0,0,0,3}","{9,9,9,9,9, 5,9,9,9,9, 257,0,0,0,0, 0,0,0,0,3}"
6,{},{},{},{},{},{},{},{},"{12,12,12,12,12,12}","{5,9,9,9,9,9}","{12,12,12,12,12,12}",{},"{5,9,9,9,9,9}","{12,12,12,12,12,12}","{12,12,12,12,12,12, 5,9,9,9,9,9}","{12,12,12,12,12,12, 5,9,9,9,9,9}"
7,{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{}
8,{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{}
9,{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{}
10,{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{}


In [47]:
# ngram set -> ngram 개수
overlap_ngram_num = pd.DataFrame(columns=["N","nginx","httpd","tomcat","node","mongodb","mysql","mariadb","redis","gcc","openjdk","gzip","bzip2","qalc","ghostscript","lowriter","total"])

for index, row in overlap_ngram.iterrows():
    overlaptmp = {"N":index, "nginx":len(row.nginx),"httpd":len(row.httpd),"tomcat":len(row.tomcat),"node":len(row.node),
                  "mongodb":len(row.mongodb),"mysql":len(row.mysql),"mariadb":len(row.mariadb),"redis":len(row.redis),
                  "gcc":len(row.gcc),"openjdk":len(row.openjdk),"gzip":len(row.gzip),"bzip2":len(row.bzip2),"qalc":len(row.qalc),
                  "ghostscript":len(row.ghostscript),"lowriter":len(row.lowriter),"total":len(row.total)}
    overlap_ngram_num = overlap_ngram_num.append(overlaptmp, ignore_index=True)

# excel로 저장
overlap_ngram_num.to_excel("overlap_ngram_num.xlsx")
overlap_ngram_num

,N,nginx,httpd,tomcat,node,mongodb,mysql,mariadb,redis,gcc,openjdk,gzip,bzip2,qalc,ghostscript,lowriter,total
0,2,3,12,10,4,33,23,11,9,34,52,9,9,51,36,72,84
1,3,1,2,4,2,11,11,2,3,17,26,13,8,17,21,41,52
2,4,0,0,0,0,2,5,0,0,5,5,3,3,5,7,9,11
3,5,0,0,0,0,2,2,0,0,3,3,0,1,3,2,4,4
4,6,0,0,0,0,0,0,0,0,1,1,1,0,1,1,2,2
5,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,11,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1


# Exploit - N-gram Range 조사

In [48]:
import pandas as pd
m_ratio = pd.DataFrame(columns=["N","Ngram","Ngram_num","Exploit","Exploit_num","Ratio"])
# b_ratio = pd.DataFrame(columns=["N","Ngram","Ngram_num","Op","Op_num","Ratio"])

In [49]:
# exploit path 정리 (list에서 str으로 변환)
m_path=dict()
for eid, pathset in eid_all_path_set.items():
    tmppath = list()
    for path in  pathset:
        tmppath.append(",".join(path))
    m_path[eid]=tmppath

In [50]:
m_ngram

,N,Ngram_syscall,num_of_exploit,EIDs
0,61,"[41, 248, 248, 248, 54, 0, 3, 5, 9, 10, 12, 13...",2,"[40871, 47170]"
1,49,"[230, 0, 3, 5, 9, 10, 12, 13, 14, 17, 56, 218,...",2,"[40871, 47170]"
2,45,"[12, 12, 5, 0, 257, 4, 12, 12, 5, 0, 257, 4, 1...",2,"[47163, 50541]"
3,43,"[9, 0, 3, 5, 9, 10, 12, 13, 14, 17, 56, 218, 2...",2,"[40616, 43127]"
4,43,"[12, 12, 5, 0, 257, 4, 12, 12, 5, 0, 257, 4, 1...",2,"[47163, 50541]"
...,...,...,...,...
473,2,"[257, 0]",16,"[15024, 15774, 3, 35370, 40616, 40871, 42275, ..."
474,2,"[8, 0]",8,"[12, 20979, 27297, 3, 43127, 45553, 50808, 9083]"
475,2,"[3, 59]",9,"[39230, 40871, 43418, 47170, 6851, 9479, 9542]"
476,2,"[8, 5]",4,"[12, 3, 43127, 50541]"


In [51]:
# 오름차순으로 정렬
m_ngram = m_ngram.sort_values(by=["N"], ascending=True)
block_ngram = dict()
block_EIDs = dict()
for N in set(m_ngram.N.to_list()):
    block_ngram[N]=set()
    block_EIDs[N]=set()
for index, row in m_ngram.iterrows():
    block_ngram[row.N].add(",".join(row.Ngram_syscall))
    block_EIDs[row.N] = block_EIDs[row.N] | set(row.EIDs)

In [62]:
# 2-gram exploit 1개 추가
block_EIDs[2] = block_EIDs[2] | {"0"}

### Range
- x <= N (of N-gram Pattern) <= y
- 7 <= x <= 65
- x <= y <= 65

In [68]:
# M(exploit) coverage 조사
##############################
# range 설정
X = 7
Y = 65
##############################

m_ratio = pd.DataFrame(columns=["N","Ngram","Ngram_num","Range_ngram","Range_ngram_num","Exploit","Exploit_num","Range_exploit","Range_exploit_num","Range_ratio"])
TOTAL=106
includedEidSet = set()
includedNgramSet = set()

for N in set(m_ngram.N.to_list()):
    if N < X: # N이 X가 될 때까지 continue
        continue
    
    tmpratio = {"N":0,"Ngram":[],"Ngram_num":0,"Range_ngram":set(),"Range_ngram_num":0,"Exploit":set(),"Exploit_num":0,"Range_exploit":set(),"Range_exploit_num":0,"Range_ratio":0}    
    tmpratio["N"] = N
    tmpratio["Ngram"] = block_ngram[N]
    tmpratio["Ngram_num"] = len(block_ngram[N])
    includedNgramSet = includedNgramSet | block_ngram[N]
    tmpratio["Range_ngram"] = includedNgramSet
    tmpratio["Range_ngram_num"] = len(includedNgramSet)
    tmpratio["Exploit"] = block_EIDs[N]
    tmpratio["Exploit_num"] = len(block_EIDs[N])
    includedEidSet = includedEidSet | block_EIDs[N]
    tmpratio["Range_exploit"] = includedEidSet
    tmpratio["Range_exploit_num"] = len(includedEidSet)
    tmpratio["Range_ratio"] = len(includedEidSet)/TOTAL*100

    # pd에 append
    m_ratio = m_ratio.append(tmpratio, ignore_index=True)
# 엑셀에 저장
m_ratio.to_excel(f"exploit_ratio_from{X}to{Y}_ascending.xlsx")
m_ratio

,N,Ngram,Ngram_num,Range_ngram,Range_ngram_num,Exploit,Exploit_num,Range_exploit,Range_exploit_num,Range_ratio
0,2,"{42,54, 272,61, 79,21, 105,59, 298,3, 3,13, 13...",163,"{42,54, 272,61, 79,21, 105,59, 298,3, 3,13, 13...",163,"{8678, 41458, 15024, 9545, 33336, 24555, 926, ...",106,"{8678, 41458, 15024, 9545, 33336, 24555, 926, ...",106,100.0
1,3,"{13,56,62, 1,1,1, 5,0,3, 8,3,3, 12,12,1, 257,3...",108,"{42,54, 272,61, 79,21, 13,56,62, 105,59, 1,1,1...",271,"{8678, 41458, 15024, 9545, 25289, 895, 33336, ...",92,"{41458, 15024, 33336, 24555, 9083, 41995, 1591...",106,100.0
2,4,"{56,247,56,247, 257,5,12,12, 102,104,135,9, 0,...",52,"{13,56,62, 298,3, 3,13, 102,107,102,107, 257,4...",323,"{8678, 41458, 15024, 895, 33336, 9083, 40839, ...",78,"{41458, 15024, 33336, 24555, 9083, 41995, 1591...",106,100.0
3,5,"{9,257,3,102,59, 5,9,9,9,9, 3,12,12,257,5, 12,...",18,"{13,56,62, 298,3, 3,13, 102,107,102,107, 257,4...",341,"{35370, 15024, 43418, 8673, 895, 33336, 40839,...",49,"{41458, 15024, 33336, 24555, 9083, 41995, 1591...",106,100.0
4,6,"{257,4,257,4,257,4, 101,61,101,101,101,101, 15...",20,"{13,56,62, 298,3, 3,13, 102,107,102,107, 257,4...",361,"{8678, 41458, 15024, 35370, 33523, 8673, 33336...",54,"{41458, 15024, 33336, 24555, 9083, 41995, 1591...",106,100.0
5,7,"{61,101,61,101,101,101,101, 101,101,101,101,10...",9,"{13,56,62, 298,3, 3,13, 102,107,102,107, 257,4...",370,"{8678, 35370, 33523, 8673, 33336, 895, 15916, ...",34,"{8678, 41458, 15024, 9545, 33336, 24555, 926, ...",106,100.0
6,8,"{101,61,101,101,101,101,101,101, 12,12,12,12,1...",8,"{13,56,62, 298,3, 3,13, 102,107,102,107, 257,4...",378,"{35370, 8673, 33336, 895, 15916, 47168, 15285,...",34,"{8678, 41458, 15024, 9545, 33336, 24555, 926, ...",106,100.0
7,9,"{257,1,3,257,1,3,257,1,3, 12,12,257,12,12,257,...",6,"{13,56,62, 22,3,3,22,3,3,22,3,3, 298,3, 3,13, ...",384,"{15024, 33336, 895, 40839, 9083, 15916, 17391,...",27,"{8678, 41458, 15024, 9545, 33336, 24555, 926, ...",106,100.0
8,10,"{61,101,61,101,101,101,101,101,101,61, 12,12,5...",6,"{13,56,62, 257,89,1,41,257,4,8,1,3,59, 22,3,3,...",390,"{35370, 15024, 33336, 15916, 47168, 15285, 422...",27,"{8678, 41458, 15024, 9545, 33336, 24555, 926, ...",106,100.0
9,11,"{12,12,257,5,0,5,0,5,0,5,0, 257,5,9,3,257,0,5,...",5,"{13,56,62, 257,89,1,41,257,4,8,1,3,59, 22,3,3,...",395,"{15023, 41994, 33516, 33336, 895, 47170, 15916...",25,"{8678, 41458, 15024, 9545, 33336, 24555, 926, ...",106,100.0


# 반복 처리

In [123]:
with open("all_patterns.txt", "w") as f:
    for index, row in m_ngram.iterrows():
        f.write(",".join(row.Ngram_syscall)+"\n")

In [104]:
# N-gram 생성 함수
def tokenize(trg, N=1):
    a = trg.split(",")                 
    d = []
    for i in range(len(a) - N+1):
        b = a[i:i+N]
        c = (',').join(b)
        d.append(c)
    return d

In [156]:
def find_pattern(ex):    
    all_pattern = set()
    # make ngram
    ngrams = set()
    for i in range(2, len(ex.split(","))):
        for token in tokenize(ex, i):
            ngrams.add(token)
    # find ngram pattern
    for ngram in ngrams:
        replaced_ex = ex.replace(ngram,"*")
        if "*,*,*" in replaced_ex:
#             print("origin sequence:", ex)
#             print("ngram:\t\t",ngram)
#             print("masked sequence:", replaced_ex)
#             print("=="*30)
            all_pattern.add(ngram)
    return  all_pattern

In [157]:
with open("all_patterns.txt") as f:
    sysList = f.readlines()
    
all_pattern = set()
for seq in sysList:
    all_pattern = all_pattern | find_pattern(seq.strip())

print(all_pattern)

{'0,3,5,9,10,12,13,14,17,56,218,257,273,302', '5,0,257,4', '233,232,232', '12,5,0,257,4,12', '16,3,44,21,21,41', '21,21,41,16,3,44', '257,4,257,4', '230,0,3,5,9,10,12,13,14,17,56,218,257,273,302,1', '321,321,1,321,321', '321,1,321,321,321', '233,232', '250,230', '53,44', '21,41,16,3,44,21', '4,257', '56,61', '0,3,5,9,10,12,13,14,17,56,218,257,273,302,1,230', '54,54', '257,4,230', '12,12', '0,257,4,5', '257,4,12,12,5,0', '257,4,5,0', '0,5', '5,0', '41,16,3,44,21,21', '12,12,5,0,257,4', '321,321,321,1,321', '5,0,5,0', '44,21,21,41,16,3', '230,257,4', '33,233,3', '1,321,321,321,321', '101,101', '230,250', '4,230,257', '0,257,4,12,12,5', '3,44,21,21,41,16', '22,3,3', '3,33,233', '321,321,321,321,1', '257,1,3', '257,4', '5,0,257,4,12,12'}


In [158]:
all_pattern_list = list(all_pattern)
all_pattern_list.sort(key=len)

all_pattern_list

['0,5',
 '5,0',
 '53,44',
 '4,257',
 '56,61',
 '54,54',
 '12,12',
 '257,4',
 '22,3,3',
 '233,232',
 '250,230',
 '5,0,5,0',
 '101,101',
 '230,250',
 '257,1,3',
 '33,233,3',
 '3,33,233',
 '5,0,257,4',
 '257,4,230',
 '0,257,4,5',
 '257,4,5,0',
 '230,257,4',
 '4,230,257',
 '233,232,232',
 '257,4,257,4',
 '12,5,0,257,4,12',
 '257,4,12,12,5,0',
 '12,12,5,0,257,4',
 '0,257,4,12,12,5',
 '5,0,257,4,12,12',
 '16,3,44,21,21,41',
 '21,21,41,16,3,44',
 '21,41,16,3,44,21',
 '41,16,3,44,21,21',
 '44,21,21,41,16,3',
 '3,44,21,21,41,16',
 '321,321,1,321,321',
 '321,1,321,321,321',
 '321,321,321,1,321',
 '1,321,321,321,321',
 '321,321,321,321,1',
 '0,3,5,9,10,12,13,14,17,56,218,257,273,302',
 '230,0,3,5,9,10,12,13,14,17,56,218,257,273,302,1',
 '0,3,5,9,10,12,13,14,17,56,218,257,273,302,1,230']

In [180]:
iter_seqNumList = dict()
for testcase in seqNumList.keys():
    print(testcase)
    seq = ",".join(seqNumList[testcase])
    for pattern in all_pattern_list:
        if pattern in seq:
#             print(pattern)
            seq = seq.replace(pattern, "*")
            if "*,*" in seq:
                for i in range(seq.count("*")):
                    seq = seq.replace("*,*","*")
                seq = seq.replace("*", pattern+","+pattern+","+pattern)
                break
            else:
                seq = seq.replace("*", pattern)
    iter_seqNumList[testcase] = seq.split(",")

gcc_strace1
gcc_strace2
httpd_get
httpd_post
httpd_put
qalc_matrix
qalc_vector
gzip_unzip
gzip_zip
mysql_c
mysql_d
mysql_r
mysql_u
mongodb_c
mongodb_d
mongodb_r
mongodb_u
mariadb_c
mariadb_d
mariadb_i
mariadb_r
mariadb_u
node_get
node_post
redis_c
redis_d
redis_r
redis_u
bzip2_unzip
bzip2_zip
ghostscript_eps2png
ghostscript_pdf2ps
ghostscript_ps2pdf
ghostscript_render
ghostscript_render2gray
tomcat_get
tomcat_post
tomcat_put
lowriter_doctopdf
lowriter_encodeUTF8
lowriter_odttopdf
nginx_get
nginx_post
nginx_put
openjdk_strace1
openjdk_strace2
